In [ ]:
!pip install tensorflow-addons
!pip install tensorflow==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 40.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
 

In [ ]:
!pip install --upgrade adapt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 26.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 20.2 MB/s eta 0:00:00
  Created wheel for adapt: filename=adapt-0.4.4-py3-none-any.whl size=131789 sha256=1a7877673cf776b552e832d450c0ce5009ec903dfa0b278bcf37739968fd163a
  Stored in directo

In [ ]:
!pip install scikeras
!pip install scikeras[tensorflow]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
import os




In [ ]:
del test_df
del train_df




In [ ]:

df = pd.read_csv("/content/train.csv.zip")
test_df = pd.read_csv("/content/test.csv")




TypeError: 'str' object is not callable

In [ ]:
# 2. Handle Mileage:
df['milage'] = df['milage'].str.replace(' mi.', '').str.replace(',', '').astype(float)

In [ ]:
X = df.drop('price', axis=1)
y = df['price']


In [ ]:
numerical_cols = ['milage']
categorical_cols = ['brand', 'model', 'model_year', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# 5. Create ColumnTransformer
transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Keep other columns as is if there are any
)

In [ ]:
transformer.fit(X)
X_transformed = transformer.transform(X)
input_dim = X_transformed.shape[1]

# 6. Function to build the model
def build_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),  # Add dropout to prevent overfitting
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

keras_regressor = KerasRegressor(build_fn=build_model, input_dim=3242, epochs=100, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1)

# 8. Build the pipeline
model_pipeline = Pipeline(
    steps=[
        ('transformer', transformer),
        ('model', keras_regressor)
    ]
)

# 10. Fit the model
model_pipeline.fit(X_train, y_train)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 4919078912.0000 - val_loss: 4327504384.0000
Epoch 2/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 5126280192.0000 - val_loss: 4274810880.0000
Epoch 3/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4831058432.0000 - val_loss: 4115709696.0000
Epoch 4/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3566578176.0000 - val_loss: 3805488128.0000
Epoch 5/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3942276096.0000 - val_loss: 3384994560.0000
Epoch 6/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3692756480.0000 - val_loss: 2932910592.0000
Epoch 7/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2290931968.0000 - val_loss: 2566830336.0000
Epoch 8/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2733101824.0000 - val_loss: 2327035648.0000
Epoch 9/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2574498816.0000 - val_loss: 2193580032.0000
Epoch 10/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2468495616.0000 - val_lo

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['milage']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['brand', 'model',
                                                   'model_year', 'fuel_type',
                                                   'engine', 'transmission',
                                                   'ext_col', 'int_col',
                                                   'accident',
                                                   'clean_title'])])),
                ('model',
                 KerasRegressor(batch_size=32, build_fn=<function build_model at 0x7ad71e1f72e0>, callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7ad71cc98f10>], epochs=100, input_dim=3242, validation_split=0.2))])

In [ ]:
keras_regressor = KerasRegressor(build_fn=build_model, input_dim=3242, epochs=100, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1)
keras_regressor.model = build_model(input_dim)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
test_df = pd.read_csv('/content/used_cars.csv')

In [ ]:
test_df['milage'] = test_df['milage'].str.replace(' mi.', '').str.replace(',', '').astype(float)


In [ ]:
test_predictions = model_pipeline.predict(test_df)


126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [ ]:
submission = pd.DataFrame({'id': test_df.index, 'price': test_predictions})
submission.to_csv('/content/submission.csv', index=False)
print("Submission file created successfully.")

Submission file created successfully.


In [ ]:
submission_file_path = '/content/submission.csv'
submission_df = pd.read_csv(submission_file_path)
submission

,id,price
0,0,15764.171875
1,1,67487.601562
2,2,50575.769531
3,3,15481.334961
4,4,106194.382812
...,...,...
4004,4004,313859.187500
4005,4005,56133.949219
4006,4006,91646.007812
4007,4007,58881.113281
